In [8]:
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import random
from sklearn.model_selection import train_test_split
import cv2
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### Coding a simple model

In [ ]:
class MLP:
    def __init__(self, neurons_in_layer, learning_rate):
        self.neurons_in_layer = neurons_in_layer   # 1 input layer, 2 hidden, 1 output layer so a 1x4 array
        self.learning_rate = learning_rate
        
        self.layers = len(neurons_in_layer) # Excluding final layer

        # Initializing weights & biases
        self.weights = []
        self.biases = []

        for i in range(self.layers - 1):
            # In the matrices rows correspond to neurons in the current layer and columns to neurons in the next layer
            self.weights.append(np.random.uniform(-1,1,size=[neurons_in_layer[i],neurons_in_layer[i+1]]))
            self.biases.append(np.zeros((1, neurons_in_layer[i+1])))
    
    def ReLU(self, x):
        return 1./(1.+np.exp(-x))
    
    def ReLU_derivative(self, x):
        return x*(1-x)
    
    # To have probabilities (all sum up to 1) in the output layer
    def softmax(self, x):
        exp_x = np.exp(x)
        return exp_x / exp_x.sum(axis=1, keepdims=True)
    
    def feed_forward(self, batch):
        self.layer_output = [batch]

        # Passing through each layer except the last one
        for i in range(self.layers - 1):
            output = self.layer_output[-1].dot(self.weights[i]) + self.biases[i]   # weight * x + bias
            self.layer_output.append(self.ReLU(output))

        # Using softmax in the output layer
        self.layer_output[-1] = self.softmax(self.layer_output[-1])
        return self.layer_output[-1]
        #final_x = np.dot(self.layer_output[-1], self.weights[-1]) + self.biases[-1]
        #self.x_values.append(final_x)
        #self.layer_output.append(self.softmax(final_x))

    def backpropagation(self, y):
        output_error = self.layer_output[-1] - y  # cross- entropy loss
        deltas = [output_error]

        # Backpropagation through each layer backwards (duh)
        for i in range(self.layers - 3, -1, -1):
            delta = deltas[-1].dot(self.weights[i+1].T) * self.ReLU_derivative(self.layer_output[i+1])
            deltas.append(delta)

        # Reversing deltas to match layer order
        deltas.reverse()

        # Updating weights and biases
        for i in range(self.layers - 1):
            self.weights[i] -= self.learning_rate * self.layer_output[i].T.dot(deltas[i])
            self.biases[i] -= self.learning_rate * np.sum(deltas[i], axis=0, keepdims=True)

    def train(self, X, y, epochs = 1000):
        for epoch in range(epochs):
            output = self.feed_forward(X)
            self.backpropagation(y)
            print(output)
            if epoch % 100 == 0:
                loss = -np.sum(y * np.log(output + 1e-9)) / X.shape[0]
                print(f'Epoch {epoch}, Loss: {loss}')

    def predict(self, X):
        output = self.feed_forward(X)
        print("Output:", output)
        print("Label:", np.argmax(output, axis=1))
        return np.argmax(output, axis=1)

    

### Loading data

In [4]:
data_file = "C:/Users/afrod/Documents/Neural_Networks/MergedDataset"
classes = ["NonDemented", "VeryMildDemented", "MildDemented", "ModerateDemented"]
training_data = []


def create_training_data():
    for dementia_level in classes:
        path = os.path.join(data_file, dementia_level)
        class_num = classes.index(dementia_level)
        for img in os.listdir(path):
            # Convert to grayscale for smaller array dimensions
            img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
            final_array = cv2.resize(img_array, (50,47))
            training_data.append([final_array, class_num])

create_training_data()

In [ ]:
random.shuffle(training_data)

# Separating features and labels
# Images are also flattened to be used as input in the knn algorithm
X = np.array([features for features, _ in training_data]).reshape(-1, 50*47)
y = np.array([label for _, label in training_data])

# Rescaling
X = X/X.max()

# One-hot encoding
y_onehot = np.zeros((y.size, int(y.max()) + 1))
y_onehot[np.arange(y.size),y.astype(int)] = 1.0

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.4, random_state=42)

40384


### Training time

In [17]:
# Define layer sizes for a network with multiple hidden layers
neurons_in_layer = [2350, 700, 100, 4]  # 784 input, two hidden layers, 4 possible classes
mlp = MLP(neurons_in_layer=neurons_in_layer, learning_rate=0.01)
mlp.train(X_train, y_train, epochs=10)

[[0.17808434 0.20538655 0.35662203 0.25990708]
 [0.22604515 0.16548928 0.35120107 0.25726449]
 [0.15109094 0.17108427 0.36611508 0.31170971]
 ...
 [0.21837117 0.18804728 0.41893405 0.1746475 ]
 [0.15868902 0.16181326 0.35363458 0.32586313]
 [0.16864331 0.20599493 0.38820453 0.23715723]]
Epoch 0, Loss: 1.4968724058753025


C:\Users\afrod\AppData\Local\Temp\ipykernel_3336\2800445351.py:18: RuntimeWarning: overflow encountered in exp
  return 1./(1.+np.exp(-x))


[[0.36552929 0.36552929 0.13447071 0.13447071]
 [0.36552929 0.36552929 0.13447071 0.13447071]
 [0.36552929 0.36552929 0.13447071 0.13447071]
 ...
 [0.36552929 0.36552929 0.13447071 0.13447071]
 [0.36552929 0.36552929 0.13447071 0.13447071]
 [0.36552929 0.36552929 0.13447071 0.13447071]]
[[0.36552929 0.13447071 0.36552929 0.13447071]
 [0.36552929 0.13447071 0.36552929 0.13447071]
 [0.36552929 0.13447071 0.36552929 0.13447071]
 ...
 [0.36552929 0.13447071 0.36552929 0.13447071]
 [0.36552929 0.13447071 0.36552929 0.13447071]
 [0.36552929 0.13447071 0.36552929 0.13447071]]
[[0.1748777  0.47536689 0.1748777  0.1748777 ]
 [0.1748777  0.47536689 0.1748777  0.1748777 ]
 [0.1748777  0.47536689 0.1748777  0.1748777 ]
 ...
 [0.1748777  0.47536689 0.1748777  0.1748777 ]
 [0.1748777  0.47536689 0.1748777  0.1748777 ]
 [0.1748777  0.47536689 0.1748777  0.1748777 ]]
[[0.47536689 0.1748777  0.1748777  0.1748777 ]
 [0.47536689 0.1748777  0.1748777  0.1748777 ]
 [0.47536689 0.1748777  0.1748777  0.17487

In [18]:
# Evaluate accuracy on test set
predictions = mlp.predict(X_train)
y_test_labels = np.argmax(y_train, axis=1)
accuracy = np.mean(predictions == y_test_labels)
#print(predictions)
#print("\nMLP with layers [2350, 700, 100, 4], lr = 0.01, epochs = 1000:")
#print(f'Test Accuracy: {accuracy * 100:.2f}%')
#print("Accuracy:", accuracy_score(y_test_labels, predictions))
#print("Classification Report:\n", classification_report(y_test_labels, predictions))
#print("Confusion Matrix:\n", confusion_matrix(y_test_labels, predictions))

C:\Users\afrod\AppData\Local\Temp\ipykernel_3336\2800445351.py:18: RuntimeWarning: overflow encountered in exp
  return 1./(1.+np.exp(-x))


Output: [[0.29692274 0.29692274 0.29692274 0.10923177]
 [0.29692274 0.29692274 0.29692274 0.10923177]
 [0.29692274 0.29692274 0.29692274 0.10923177]
 ...
 [0.29692274 0.29692274 0.29692274 0.10923177]
 [0.29692274 0.29692274 0.29692274 0.10923177]
 [0.29692274 0.29692274 0.29692274 0.10923177]]
Label: [0 0 0 ... 0 0 0]
